<a href="https://colab.research.google.com/github/pwskym1234/admin/blob/main/%EC%B1%97%EB%B4%87%EB%A7%8C%EB%93%A4%EA%B8%B0_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 해당 노트북을 꼭 복사해서 사용해 주세요. 이 노트북은 공개 노트북이기 때문에 키값 등이 유출될 수 있습니다.

In [ ]:
# 관련 라이브러리 설치하기

In [ ]:
!pip install langchain langchain_openai pypdf chromadb

In [ ]:
# 관련 모듈 import 하기

In [ ]:
import os
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
# OpenAI API Key 세팅

In [ ]:
os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"

실습을 위해 PDF 파일 다운로드 받기

In [ ]:
loaders = [
    PyPDFLoader("docs/카이스트미래전략2024_본문1부1장.pdf"),
    PyPDFLoader("docs/카이스트미래전략2024_본문1부2장.pdf"),
]

In [ ]:
docs = []
for loader in loaders:
    docs.extend(loader.load())

In [ ]:
# chunk_size는 자르고자 하는 글자수, chunk_overlap 은 각 부분마다 겹쳐도 되는 글자 수
# Recursive Character Text Splitter 는 최대한 관련 있는 문장들을 같이 묶어서 잘라주는 기능을 한다.
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=0,
)
splits = text_splitter.split_documents(docs)

embedding = OpenAIEmbeddings()
persist_directory = "docs/chroma/"

# vectordb에 저장하기
vectordb = Chroma.from_documents(
    documents=splits, embedding=embedding, persist_directory=persist_directory
)
vectordb.persist()
print(vectordb._collection.count(), "documents in collection saved.")

91 documents in collection saved.


In [ ]:
# 임베딩을 하면 chroma DB (벡터 데이터베이스)에 저장완료!

In [ ]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
llm.invoke('Hello, how are you?')

AIMessage(content="Hello! I'm just a computer program, so I don't have feelings, but I'm here to help you. How can I assist you today?")

In [ ]:
db = Chroma(persist_directory=persist_directory, embedding_function=embedding)
retriever = db.as_retriever()
docs = retriever.get_relevant_documents("미래에서 가장 중요한 것은 무엇인가?")
print(docs)
print(len(docs), "개의 문서가 리트리벌되었습니다.")

[Document(page_content='1 포스트 AI 시대 휴머니즘의 미래   31을 만드는 글로벌 빅테크 기업이고, 아이폰에는 AI 시리 Siri가 탑재돼 있\n다. 2023년 6월에는 세계개발자회의에서 VR(가상현실) 과 AR(증강현실) 을 \n통합한 MR(복합 현실)  헤드셋 ‘비전 프로’를 선보여 혁신적인 기술력을 \n과시했다. 말하자면 애플은 디지털전환 시대에 걸맞은 선도적 디지털 \n도구를 만들고 있는 동시에 도구의 기술적 진화를 통해 시대의 변화를 \n견인하고 있는 셈이다. \n인류의 진화는 이렇듯 늘 도구와 함께해왔고, 인류 역사는 곧 도구의 \n발명과 진화의 역사이기도 하다. 또 인간은 도구를 통해 인간의 감각과 \n경험을 확장해왔다. 현시점에서 가장 진화한 도구라고 할 수 있는 AI가 \n이제 인간의 생각과 사상을 확장해갈, 완전히 새로운 도구의 시대를 열\n고 있다.', metadata={'page': 14, 'source': 'docs/카이스트미래전략2024_본문1부1장.pdf'}), Document(page_content='1 포스트 AI 시대 휴머니즘의 미래   23이나 취향과도 관련이 있다.\n21세기 디지털전환 시대를 상징하는 대표적인 도구로는 스마트폰과 \nAI, 다양한 애플리케이션 등을 들 수 있다. 19세기를 살았던, 《브리야사\n바랭의 미식 예찬》의 저자이자 프랑스의 미식가였던 브리야사바랭 Bril-\nlat-Savarin은 “당신이 뭘 먹었는지 말해주면 당신이 어떤 사람인지 말해주\n겠다”라고 말했다. 21세기에는 “당신이 어떤 스마트기기와 애플리케이\n션, AI를 사용하는지 말해주면 당신이 어떤 사람인지 말해주겠다”라고 \n말해도 무방할 것이다.1\n도구의 역사\n19세기 초 덴마크의 고고학자 크리스티안 위르겐센 톰센 Christian  Jürgensen  \nThomsen은 문자로 기록되기 전인 선사 先史시대를 연구하면서 고고학 유\n물이 제작된 연대 비교를 통해 유물의 핵심 물질이 돌, 청동, 철임을 밝\n혀냈다

In [ ]:
template = """Answer the question based only on the following context:

{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI()

def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])

chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [ ]:
answer = chain.invoke("바이오닉맨이란?")
print(answer)

바이오닉맨은 생체공학 기술인 바이오닉스를 활용하여 인간의 일부를 기계적 부품으로 대체함으로써 장애를 극복하거나 인간의 능력을 향상시키는데 주목하는 개념이다.


In [ ]:
# 다양한 검색어 입력해 보기
# 세상을 바꾼 101가지 발명품 1. 25p
# 바이오닉맨이란? 2. 72p